# Data Engineering 

Given the date and country of each transaction, use the API to
determine whether it was a national holiday

Data information: 
- Dataset: https://archive.ics.uci.edu/ml/datasets/Online+Retail#
- REST API: https://holidayapi.com/

- API KEY: *c88cae9e-bee1-4b0e-871f-ff60cb63a115*



## Import libraries

In [1]:
import requests
import urllib.request
import pandas as pd
import csv
from urllib.request import urlretrieve
import datetime
   

## Functions / Constants Definitions

In [2]:
def url_builder_holidays(country, year):
    
    user_api = 'c88cae9e-bee1-4b0e-871f-ff60cb63a115'   # Get it from https://holidayapi.com/
    base_url = 'https://holidayapi.com/v1/holidays'     # base API GET URL

    full_api_url = base_url +'?'+'key=' + user_api + '&country=' + country + '&public=True'  \
                    + '&year=' + str(year) + '&format=csv'
    print(full_api_url)
    return full_api_url


def data_fetch_holidays(country, year):
    
    full_api_url = url_builder_holidays(country, year)
           
    try:
        df = pd.read_csv(full_api_url)
        df['country'] = country
        df['year'] = year
        df = df[['country', 'year','date','name']] 
        status = 200
        
    except:
        print('KO--> error retreiving data')
        df = []
        status = 400  
        pass
    
    return status, df

path_data = 'data/'

In [3]:
# Test get info from API holidays

data_retrieve = data_fetch_holidays('US',2017)
print('Status:',data_retrieve[0])
print('DatFarme:',data_retrieve[1])

https://holidayapi.com/v1/holidays?key=c88cae9e-bee1-4b0e-871f-ff60cb63a115&country=US&public=True&year=2017&format=csv
Status: 200
DatFarme:    country  year        date                          name
0       US  2017  2017-01-01                New Year's Day
1       US  2017  2017-01-16   Martin Luther King, Jr. Day
2       US  2017  2017-02-20  George Washington's Birthday
3       US  2017  2017-05-29                  Memorial Day
4       US  2017  2017-07-04              Independence Day
5       US  2017  2017-09-04                     Labor Day
6       US  2017  2017-10-09                  Columbus Day
7       US  2017  2017-11-11                  Veterans Day
8       US  2017  2017-11-23              Thanksgiving Day
9       US  2017  2017-12-25                 Christmas Day
10      US  2017  2017-12-31                New Year's Eve


## Import data from Retail Stores

In [4]:
xl = pd.read_excel(path_data+'Online Retail.xlsx')

In [5]:
# Analyze dataframe structure
xl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      541909 non-null object
StockCode      541909 non-null object
Description    540455 non-null object
Quantity       541909 non-null int64
InvoiceDate    541909 non-null datetime64[ns]
UnitPrice      541909 non-null float64
CustomerID     406829 non-null float64
Country        541909 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [6]:
# Sample of number of records x country 
xl['Country'].value_counts().sample(10)

France          8557
Sweden           462
EIRE            8196
Malta            127
Norway          1086
Saudi Arabia      10
Cyprus           622
Denmark          389
Austria          401
USA              291
Name: Country, dtype: int64

In [7]:
# Generate a frame with countries descriptions from sales file

xl_country = xl[['Country']].drop_duplicates()
print(xl_country.shape)

(38, 1)


In [8]:
# Url to get country codes, format: ISO 2-digit code from ISO 3166-alpha-2 

url = 'https://pkgstore.datahub.io/core/country-list/data_csv/data/d7c9d7cfb42cb69f4422dec222dbbaa8/data_csv.csv'
urlretrieve(url,'data/data_csv.csv')

country_codes = pd.read_csv(path_data+'data_csv.csv')
country_codes.sample(5) 

,Name,Code
217,"Taiwan, Province of China",TW
32,British Indian Ocean Territory,IO
203,Solomon Islands,SB
86,Greenland,GL
89,Guam,GU


In [9]:
# join country codes with contries of the original dataset and check shapes

xl_country = xl_country.set_index('Country').join(country_codes.set_index('Name'))

print(xl_country.shape)

xl_country.sample(5)

(38, 1)


,Code
Country,
Bahrain,BH
Saudi Arabia,SA
Finland,FI
EIRE,NaN
France,FR


In [10]:
# Detect countries with null code null values and update it manually

xl_country[xl_country['Code'].isnull()]

,Code
Country,
EIRE,NaN
Channel Islands,NaN
Unspecified,NaN
USA,NaN
European Community,NaN
RSA,NaN


In [11]:
# Set up codes with more than two letters for configuration of territories 
# predefined in origina sales data

xl_country.loc['EIRE']['Code'] = 'IE'                 # Ireland
xl_country.loc['Channel Islands']['Code'] = 'CHAI'    # Channel Islands
xl_country.loc['Unspecified']['Code'] = 'UNDEF'       # Undefined
xl_country.loc['USA']['Code'] = 'US'                  # United States of America
xl_country.loc['European Community']['Code'] = 'EURC' # Global European Community
xl_country.loc['RSA']['Code'] = 'ZA'                  # South Africa
print(xl_country[xl_country['Code'].isnull()].shape)

(0, 1)


In [13]:
# Rename the column Country, join data with original df and check shapes

xl_country.columns = ['Country_Code']

xl2 = xl.set_index('Country').join(xl_country).reset_index()
print(xl2.shape)
xl2.sample(5)



(541909, 9)


,Country,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country_Code
99645,United Kingdom,541136,22727,ALARM CLOCK BAKELIKE RED,6,2011-01-14 10:39:00,3.75,18229.0,GB
334830,United Kingdom,564754,23339,EGG FRYING PAN BLUE,1,2011-08-30 10:29:00,4.13,NaN,GB
8266,EIRE,555650,22844,VINTAGE CREAM DOG FOOD CONTAINER,2,2011-06-06 13:48:00,8.50,14156.0,IE
103337,United Kingdom,541424,21270,ANTIQUE CREAM CUTLERY CUPBOARD,2,2011-01-17 17:57:00,8.29,NaN,GB
413707,United Kingdom,571671,22904,CALENDAR PAPER CUT DESIGN,6,2011-10-18 13:15:00,2.95,16940.0,GB


## Retrieve information from the API hoildays

In [14]:
# Retrieve Data from api for every country and year defined

year_intervals = [2010,2011,2012]

df_holidays = pd.DataFrame(columns=['country','year','date','name'])

for country_code in xl2['Country_Code'].unique():
    print(country_code)
    
    if len(country_code) == 2:       # assume country codes with length greater tham 2 are other territories config.
        
        for year in year_intervals:
            print(year)
            
            data_retrieve = data_fetch_holidays(country_code,year)
            
            if data_retrieve[0] == 200:  # status code was ok
                df_holidays = df_holidays.append(data_retrieve[1])
        
        

AU
2010
https://holidayapi.com/v1/holidays?key=c88cae9e-bee1-4b0e-871f-ff60cb63a115&country=AU&public=True&year=2010&format=csv
2011
https://holidayapi.com/v1/holidays?key=c88cae9e-bee1-4b0e-871f-ff60cb63a115&country=AU&public=True&year=2011&format=csv
2012
https://holidayapi.com/v1/holidays?key=c88cae9e-bee1-4b0e-871f-ff60cb63a115&country=AU&public=True&year=2012&format=csv
AT
2010
https://holidayapi.com/v1/holidays?key=c88cae9e-bee1-4b0e-871f-ff60cb63a115&country=AT&public=True&year=2010&format=csv
2011
https://holidayapi.com/v1/holidays?key=c88cae9e-bee1-4b0e-871f-ff60cb63a115&country=AT&public=True&year=2011&format=csv
2012
https://holidayapi.com/v1/holidays?key=c88cae9e-bee1-4b0e-871f-ff60cb63a115&country=AT&public=True&year=2012&format=csv
BH
2010
https://holidayapi.com/v1/holidays?key=c88cae9e-bee1-4b0e-871f-ff60cb63a115&country=BH&public=True&year=2010&format=csv
KO--> error retreiving data
2011
https://holidayapi.com/v1/holidays?key=c88cae9e-bee1-4b0e-871f-ff60cb63a115&country

KO--> error retreiving data
MT
2010
https://holidayapi.com/v1/holidays?key=c88cae9e-bee1-4b0e-871f-ff60cb63a115&country=MT&public=True&year=2010&format=csv
2011
https://holidayapi.com/v1/holidays?key=c88cae9e-bee1-4b0e-871f-ff60cb63a115&country=MT&public=True&year=2011&format=csv
2012
https://holidayapi.com/v1/holidays?key=c88cae9e-bee1-4b0e-871f-ff60cb63a115&country=MT&public=True&year=2012&format=csv
NL
2010
https://holidayapi.com/v1/holidays?key=c88cae9e-bee1-4b0e-871f-ff60cb63a115&country=NL&public=True&year=2010&format=csv
2011
https://holidayapi.com/v1/holidays?key=c88cae9e-bee1-4b0e-871f-ff60cb63a115&country=NL&public=True&year=2011&format=csv
2012
https://holidayapi.com/v1/holidays?key=c88cae9e-bee1-4b0e-871f-ff60cb63a115&country=NL&public=True&year=2012&format=csv
NO
2010
https://holidayapi.com/v1/holidays?key=c88cae9e-bee1-4b0e-871f-ff60cb63a115&country=NO&public=True&year=2010&format=csv
2011
https://holidayapi.com/v1/holidays?key=c88cae9e-bee1-4b0e-871f-ff60cb63a115&country

In [15]:
print(df_holidays.info())
df_holidays.sample(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1014 entries, 0 to 7
Data columns (total 4 columns):
country    1014 non-null object
year       1014 non-null object
date       1014 non-null object
name       1014 non-null object
dtypes: object(4)
memory usage: 39.6+ KB
None


,country,year,date,name
1,AT,2012,2012-01-06,Epiphany
2,ZA,2012,2012-04-06,Good Friday
2,SG,2012,2012-01-24,Second day of Chinese New Year
3,HK,2011,2011-02-05,Lunar New Year - Day 3
5,DK,2010,2010-04-30,Store bededag


In [17]:
# Save this info to use it in other company processes

file = path_data + 'holidays.gz'
print(file)
df_holidays.to_csv(file, sep='|', encoding='utf-8', compression='gzip', index_label='idx')

data/holidays.gz


In [18]:
df_holidays['date'] = pd.to_datetime(df_holidays['date'], format='%Y-%m-%d')

df_holidays = df_holidays[['country','date']]  \
    .groupby(['country','date']) \
    .last() \
    .reset_index()

df_holidays['holiday'] = True

df_holidays.columns = ['Country_Code','Date','Holiday']
df_holidays.sample(5)

,Country_Code,Date,Holiday
95,BE,2012-05-28,True
858,SE,2010-01-01,True
336,FI,2012-04-09,True
550,IL,2011-12-22,True
318,FI,2010-12-25,True


## Join holidays with transactions

In [19]:
xl2['Date'] =  xl2['InvoiceDate'].dt.floor('d')

xl3 = xl2.copy()

xl3 = xl3.set_index(['Country_Code','Date']) \
         .join(df_holidays.set_index(['Country_Code','Date'])) \
         .reset_index()

xl3['Holiday'].fillna(False, inplace=True)


In [20]:
print(xl3.shape)
xl3.groupby(['Country_Code','Holiday'])[['InvoiceNo']].count()

(541909, 11)


InvoiceNo
Country_Code Holiday           
AE           False           68
AT           False          394
             True             7
AU           False         1259
BE           False         2069
BH           False           19
BR           False           32
CA           False          151
CH           False         1998
             True             4
CHAI         False          758
CY           False          622
CZ           False           30
DE           False         9169
             True           326
DK           False          389
ES           False         2520
             True            13
EURC         False           61
FI           False          695
FR           False         8171
             True           386
GB           False       495478
GR           False          146
HK           False          288
IE           False         8088
             True           108
IL           False          297
IS           False          182
IT           False          802
             True             1
JP           False          358
LB           False           45
LT           False           35
MT           False           82
             True            45
NL           False         2371
NO           False         1086
PL           False          341
PT           False         1369
             True           150
SA           False           10
SE           False          457
             True             5
SG           False          229
UNDEF        False          446
US           False          190
             True           101
ZA           False           58

## Export data

In [21]:

file = path_data + 'retail_clean.gz'
print('Export file:', file)

xl3.to_csv(file, sep='|', encoding='utf-8', compression='gzip', index_label='idx')

Export file: data/retail_clean.gz


In [22]:
!date

sáb sep 29 09:46:00 CEST 2018
